In [2]:
!pip install azureml-sdk


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Importing the dependencies**

In [3]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [6]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]

In [7]:
print(resource_group)
print(workspace_name)
print(region)

ml-model-deployment-rg
deploy-sklearn-model-ws1
Central India


**Create a Resource Group from Azure Portal**

In [8]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Deploying Workspace with name deploy-sklearn-model-ws1.
Deploying AppInsights with name deployskinsightsf199d576.
Deployed AppInsights with name deployskinsightsf199d576. Took 107.51 seconds.
Deployed Workspace with name deploy-sklearn-model-ws1. Took 21.09 seconds.
Workspace deploy-sklearn-model-ws1 created


In [9]:
# Specify the path to your  model file
model_path = 'diabetes_model.pkl'

In [10]:
model_name='diabetes_prediction_model'

In [11]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model diabetes_prediction_model


In [12]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [13]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [18]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [19]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\Basvraj\AppData\Local\Temp\ipykernel_1920\3861565384.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-12-11 12:08:25+05:30 Creating Container Registry if not exists..
2024-12-11 12:18:26+05:30 Registering the environment.
2024-12-11 12:18:30+05:30 Building image..
2024-12-11 12:29:45+05:30 Generating deployment configuration.
2024-12-11 12:29:47+05:30 Submitting deployment to compute..
2024-12-11 12:29:56+05:30 Checking the status of deployment diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [20]:
scoring_uri = service.scoring_uri

In [21]:
scoring_uri

'http://b40ce2f4-0521-468e-896e-b5193c685731.centralindia.azurecontainer.io/score'

**Cleaning up all the created resources**

In [22]:
# Delete the service

service_name = "diabetes-prediction-service"

service = ws.webservices[service_name]
service.delete()

Running
2024-12-11 15:42:54+05:30 Check and wait for operation (156c942c-eeba-4a39-9a9c-bf775a650dfe) to finish.
2024-12-11 15:42:59+05:30 Deleting service entity.
Succeeded


In [25]:
# Specify the name of the registered model to delete
model_name = 'diabetes_prediction_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()


ModelErrorResponseException: Workspace not found.

In [26]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)

ProjectSystemException: ProjectSystemException:
	Message: Workspace not found.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Workspace not found."
    }
}

1. Azure Account with appropriate subscription
2. Configuration file that has subscription_id and other details like resource_group name, etc
3. Create a resource group from Azure Portal
4. Create a workspace
5. Register the model in container registry
6. Deploy the model as endpoint
7. Once done, delete the created resources